# $$Procesamiento\;de\;información$$

# Tarea 7. Detección de emociones
## Asher Yoav Luna Osorio

## Introducción

El análisis de sentimiento o sentiment analysis consiste en evaluar las emociones, actitudes y opiniones. Las organizaciones utilizan este método para obtener información que les permita comprender la forma en la que los clientes reaccionan respecto a un producto o servicio específico.

La herramienta de análisis de sentimiento utiliza tecnologías avanzadas de inteligencia artificial, como procesamiento del lenguaje natural, análisis de texto y ciencia de datos para identificar, extraer y estudiar información subjetiva. De esta forma se realiza la categorización para definir el sentido de la información. Específicamente para este trabajo, textos agresivos o no agresivos.

## Desarrollo

Para este ejercicio se realiza el entrenamiento de un sistema que nos ayudará a clasificar comentario en Twitter y definir si incitan a la violencia y la discriminación o no. Los textos se procesan para eliminar usuarios, hastags, urls's, signos de puntuación y emoticonse

In [1]:
import json
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer  
import numpy as np
import pandas as pd
import unicodedata
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import LinearSVC
from tabulate import tabulate

In [2]:
PUNCTUACTION = ";:,.\\-\"'/"
SYMBOLS = "()[]¿?¡!{}~<>|"
SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

def normalize_text(input_str,
                   punct=False,
                   accents=False,
                   max_dup=2):
    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not punct:
            if c in SKIP_SYMBOLS:
                continue
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    return "".join(n_str)

def line_iterator(filename):
    if filename.endswith(".gz"):
        f = gzip.GzipFile(filename)
    else:
        f = open(filename, encoding='utf8')

    while True:
        line = f.readline()
        # Test the type of the line and encode it if neccesary...
        if type(line) is bytes:
            line = str(line, encoding='utf8')

        # If the line is empty, we are done...
        if len(line) == 0:
            break

        line = line.strip()
        # If line is empty, jump to next...
        if len(line) == 0:
            continue

        yield line

    # Close the file...
    f.close()

def tweet_iterator(filename):
    for line in line_iterator(filename):
        yield json.loads(line)

In [3]:
def load_corpus(file):
    X=[]
    Y=[]
    for review in tweet_iterator(file):
        text = normalize_text(review['text'])
        X.append(text.lower())
        Y.append(review['klass'])
    return X,Y

In [4]:
ruta="C:/Users/Luna/Downloads/Maestria/Procesamiento de información/Unidad 6-7/Tarea 7/"
training="AggressivenessDetection_train.json"
prediction="AggressivenessDetection_predict.json"
X, Y= load_corpus(ruta+training)
data, agresive = load_corpus(ruta+prediction)

In [5]:
vectorizer = CountVectorizer(stop_words=stopwords.words('spanish'))  
X_vec_tf = vectorizer.fit_transform(X)

In [6]:
svc = LinearSVC(random_state=0, max_iter=3000)
svc.fit(X_vec_tf, Y)
svc_predict = svc.predict(X_vec_tf)

## Realizar predicción de datos

In [7]:
data_vec_tf1 = vectorizer.transform(data)

In [8]:
#Set de datos SVM
svc_predict_tf = svc.predict(data_vec_tf1)

In [9]:
clasificar=[]
for j in range(0,25):
    D=(data[j],svc_predict_tf[j])
    clasificar.append(D)

## Visualización de clasificación

Se visualiza una muestra de 25 Tweets y su clasificación.<br>
<br>
$\quad$ \* 0=No agresivo<br>
$\quad$ \* 1=Agresivo

In [10]:
data_frames = pd.DataFrame(clasificar, columns=['Tweet','Clasificación'])
data_frames = data_frames.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
data_frames.set_properties(**{'text-align': 'center'}).hide(axis='index')

Tweet,Clasificación
perra estamos en mexico al menos en alguna lengua nativa hubieras tuiteado inche gorda,1
muchas felicidades mi reyna que la pases bien en tu dia de las putas,0
apenas termine mi tarea y me levanto en 4 horas me lleva la verga,0
inconscientemente me hice dependiente de dos personas ahora mira tu si soy el puto amo,0
@usuario @usuario @usuario ese conchadesumadre del yerko que ni siquiera le alcanza pa maricon porque hay que ser hombre antes y a ese le alcanza pa la mitad apenas,1
y si en tus putos 16anos de vida no as sido que se hace,0
apoyemos a nuestras mujeres no puede seguir habiendo tanta inseguridad tambien #micasaestucasahermana gordas y feas abstenerse,0
lo unico que quiero es que me dejen en paz con las putas indirectas e insinuaciones,0
un dia mas de sonar que conozco a @usuario y despertar y darme cuenta que pues ni madres solo era otro sueno,0
httpstco4ytvgqgfij y llora el marica por tanto crimen que el mismo propicia,0


## Conteo y limpieza de datos

Eliminación de URL's, Hashtags, Usuarios, signos de puntuación, 

In [11]:
#Se cargan las librerias y se crean los datos con los símbolos a eliminar
from collections import Counter
import string
import re

_SYMBOLS = set(";:,.\\-\"'/()[]¿?¡!{}~<>|«»-—’\t\n\r")
quitar=string.punctuation
STOPWORDS_ES = stopwords.words("spanish")

In [12]:
ruta = pd.read_json("C:/Users/Luna/Downloads/Maestria/Procesamiento de información/Unidad 6-7/Tarea 7/AggressivenessDetection_predict.json", lines=True)

In [13]:
#Normalizar Text
def normalize_text(text,
                   accents=False,
                   max_dup=2):

    nfkd_f = unicodedata.normalize('NFKD', text)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    return (unicodedata.normalize('NFKC', "".join(n_str)))

In [14]:
#Quitar emojis
def deEmojify(x):
    regrex_pattern = re.compile(pattern= "["
                               u"\U00000000\U00000039"
                               u"\U00000100-\U000E0067"#u"\U00002660-\U0001F972"
                                   "]", flags=re.UNICODE)
    return regrex_pattern.sub(r'',x)

In [15]:
#Remover StopWords en español
def filter_words_es(text, remove_stopwords=True):
    wlist = text.split()
    o_text = []
    # filtrado de palabras
    for t in wlist:
        if remove_stopwords and t in STOPWORDS_ES:
            continue
        if t.isnumeric():
            continue
        o_text.append(t)
    return " ".join(o_text)

In [16]:
#Aplica los filtros ya inicializados
def limpieza(twitts):
    nuevo=[]
    for twitt in twitts:
        modificacion=re.sub(r'@\w+','', twitt) #elimina usuarios
        modificacion= re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', modificacion) #elimina URLs
        modificacion= re.sub(r'#\w+', '', modificacion) #elimina Hashtags
        modificacion=modificacion.lower() #cambia a minúsculas
        modificacion=normalize_text(modificacion) #normaliza el texto
        modificacion=filter_words_es(modificacion)
        for caracter in quitar:
            modificacion=modificacion.replace(caracter,
                         "")
            for caracter in _SYMBOLS:
                modificacion=modificacion.replace(caracter,
                             "")
        modificacion=deEmojify(modificacion) #elimina emoticones
        nuevo.append(modificacion)
    return nuevo

In [17]:
docs=limpieza(ruta["text"])

In [18]:
pos=[]
neg=[]
for num in range(0,len(docs)):
    if svc_predict_tf[num]==0:
        pos.append(docs[num])
    else:
        neg.append(docs[num])

In [19]:
def clasificar(datos):
    conjunto=[]
    data_frame=[]
    for twit in datos:
        for word in twit.split():
            conjunto.append(word)
            
    conteo=Counter(conjunto)
    sortByValue={k: v for k, v in sorted((conteo.items()), key= lambda v: v[1], reverse=True)}
    for i in range (0,50):
        for value in sortByValue:
            par=(value,sortByValue[value])
            data_frame.append(par)
    return data_frame

In [20]:
words_neg=clasificar(neg)
words_pos=clasificar(pos)
matrix=[]
for i in range(1,51):
    matrix.append(i)
col1 = ['Palabra Agresiva', 'Cantidad de repeticiones']
col2 = ['Palabra No agresiva','Cantidad de repeticiones']

In [21]:
df_ag = pd.DataFrame((words_neg[:50]), columns=col1, index=matrix)
df_nag = pd.DataFrame((words_pos[:50]), columns=col2, index=matrix)

# Tabla de Palabras agresivas

In [22]:
df_ag

,Palabra Agresiva,Cantidad de repeticiones
1,putos,146
2,si,86
3,madre,84
4,pinche,73
5,verga,61
6,putas,54
7,hdp,45
8,maricon,41
9,puta,40
10,mierda,33


# Tabla de Palabras agresivas

In [23]:
df_nag

,Palabra No agresiva,Cantidad de repeticiones
1,verga,283
2,si,208
3,madre,201
4,loca,170
5,putas,161
6,mas,144
7,gorda,122
8,putos,118
9,bien,94
10,solo,67


## Conclusiones

Podemos ver la clasificación realizada y las palabras que presentan mayor frecuancias en cada una de las tablas.

Observamos que en ambas tablas encontramos en diferente proporción, palabras similares. Esto debido a que la clasificación es realizada a nivel tweet y no a nivel palabra.

A continuación se muestran ejemplos de la clasificación realizada

`perra estamos en mexico al menos en alguna lengua nativa hubieras tuiteado inche gorda ---> 1`<br>
`lo unico que quiero es que me dejen en paz con las putas indirectas e insinuaciones---> 0`

Podemos ver la importancia del contexto en los textos que se analiza, ya que sin este se puede realizar una falsa clasificación y obtener información poco confiable.